# TOMOGRAF ROBI BRRR


In [1]:
import scipy
import numpy as np

from PIL import Image


## Bresenham

In [2]:
"""bychnięte z wikipedi tylko że C"""
def Bresenham(x1, y1, x2, y2):
    output_points = []
    # kierunki rysowaniaw
    xi = 1 if x1<x2 else -1
    dx = abs(x1-x2)

    yi = 1 if y1<y2 else -1
    dy = abs(y1-y2)

    x, y = x1, y1

    output_points.append((x,y))

    # oś wiodąca OX
    if dx > dy:
        ai = (dy - dx) * 2
        bi = dy * 2
        d = bi - dx
        # pętla po x-ach
        while x != x2:
            if d >= 0:
                x += xi
                y += yi
                d += ai
            else:
                d += bi
                x += xi
            output_points.append((x,y))
    # oś wiodąca OY
    else:
        ai = ( dx - dy ) * 2
        bi = dx * 2
        d = bi - dy
        # pętla po y
        while y != y2:
            if d >= 0:
                x += xi
                y += yi
                d += ai
            else:
                d += bi
                y += yi
            output_points.append((x,y))
    return output_points


print(Bresenham(1,1,10,6))


Parametry wejściowe

In [4]:
# parametry symulacji skanera 
# skok głowicy
delta_a = 1
# liczba detektorów
n = 90
# rozpiętość skanera
l = np.deg2rad(90)

PI = np.pi

In [23]:

def read_image(path):
    img = Image.open(path).convert('L')
    # arr = np.array(img.getdata())
    arr = np.array(img.getdata(), dtype=np.uint8) # Note the dtype input
    field = np.resize(arr, (img.size[1], img.size[0]))
    img = Image.fromarray(field, mode='L')
    # img.show()
    return field

test = read_image("tomograf-obrazy\kolorki-kredki-swiecowe-wykrecane-024107.jpg")
print(test)


[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [ ]:
def generate_points(r, a, n=1, fi=0):   
    points = []
    if n>1:
        for i in range(n):
            xD = r*np.cos(a + PI - fi/2 + i*(fi/(n-1)))
            yD = r*np.sin(a + PI - fi/2 + i*(fi/(n-1)))
            points.append((round(xD), round(yD)))
    else: 
        xD = r*np.cos(a + PI)
        yD = r*np.sin(a + PI)
        points.append((round(xD), round(yD)))
    return points


def generate_emiter_s(r, a, n=1, l=0):  
    emiters = generate_points(r , a, n, l)
    return emiters

def generate_detectors(r, a, n=3, l=1):  
    emiters = generate_points(r , a, n, l)
    return emiters



print(generate_points(10, 0, 2, 1))

[(-9, 5), (-9, -5)]


In [22]:
def radon_transform(photo_file, delta_a, n, l):  # zdjęcie / skok alfa / n detektorów / rozpiętość detektorów
    photo_array = read_image(photo_file)
    size = len(photo_array), len(photo_array[0])  # h , w
    radious = round(((size[0] ** 2) + (size[1] ** 2)) ** 0.5)
    
    empty = [[0 for i in range(radious*2)] for j in range(radious*2)]
    w,s = photo_array.shape
    y_off, x_off = round((r*2-w)/2), round((r*2-s)/2)
    obraz = np.array(obraz.copy())
    empty[y_off:y_off+w,x_off:x_off+s] = photo_array

    return size, radious


def radon_reversed():
    pass


print(radon_transform('tomograf-obrazy\Kropka.jpg'))

<class 'PIL.Image.Image'>
<class 'numpy.ndarray'>
((400, 400), 565.685424949238)
